## 1.Importando bibliotecas

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
from scipy.stats import chi2_contingency



## 2.Organizando os dados

In [2]:
dados = pd.read_excel('Customer-Churn.xlsx')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
dados.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7569-NMZYQ,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No
1,8984-HPEMB,Female,0,No,No,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No
2,5989-AXPUC,Female,0,Yes,No,68,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No
3,5734-EJKXG,Female,0,No,No,61,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,118.60,7365.70,No
4,8199-ZLLSA,Male,0,No,No,67,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),118.35,7804.15,Yes


In [4]:
dados['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [5]:
dados['Churn'].value_counts(normalize=True)

Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

In [6]:
dados_Churn = dados[dados['Churn'] == 'Yes']
dados_mantidos = dados[dados['Churn'] == 'No']

## 3.Analisando dados

### Analise basica de tenure

In [6]:
fig = px.histogram(dados, x='tenure', nbins=50, 
                   title='Distribuição do Tempo de Permanência')
fig.show()

### Analise de Permanência x Churn

In [8]:
# Boxplot
fig = px.box(dados, x='Churn', y='tenure',
             title='Tempo de Permanência vs Churn')
fig.show()

# Histograma comparativo
fig = px.histogram(dados, x='tenure', color='Churn', barmode='overlay',
                   title='Distribuição de Tenure: Churn vs Não-Churn')
fig.show()

- Superado os primeiros meses, a tendencia é de fidelização.

### Analise taxa de Churn por faixa de tenure

In [ ]:
# Verificar o tipo atual e valores únicos
print("Tipo da coluna Churn:", dados['Churn'].dtype)
print("Valores únicos na coluna Churn:", dados['Churn'].unique())

# Converter para numérico (caso seja 'Yes'/'No' ou 'True'/'False')
dados['Churn'] = dados['Churn'].map({'Yes': 1, 'No': 0, 'True': 1, 'False': 0, True: 1, False: 0})

# Ou se já for 0/1 mas como string
dados['Churn'] = pd.to_numeric(dados['Churn'], errors='coerce')

# Agora criar as faixas de tenure
dados['tenure_group'] = pd.cut(dados['tenure'], 
                           bins=[0, 3, 6, 12, 24, 60, 100],
                           labels=['0-3m', '4-6m', '7-12m', '13-24m', '25-60m', '60m+'])

Churn_by_tenure = dados.groupby('tenure_group')['Churn'].mean().reset_index()

fig = px.bar(Churn_by_tenure, x='tenure_group', y='Churn',
             title='Taxa de Churn por Faixa de Tempo de Permanência')
fig.show()

- 0 a 6 meses são os periodos com maior quantidade de Churns (42%).

### Periodo critico

In [ ]:
# Exemplo: "70% do nosso churn acontece nos primeiros 6 meses"
early_churn = dados[(dados['tenure'] <= 6) & (dados['Churn'] == 1)]
total_churn = dados[dados['Churn'] == 1]
early_churn_ratio = len(early_churn) / len(total_churn)
print(f"{early_churn_ratio:.1%} do churn acontece nos primeiros 6 meses")

- Como 42% dos Churns acontecem nos primeiros 6 meses.
- Focar campanhas de melhorias nessa faixa de tempo.
- Melhorar experiencia inicial, criar motivos para permanencia como programas de fidelidades.

### Analise de genero

In [ ]:
# Gráfico de barras agrupadas
fig = px.histogram(dados, x='gender', color='Churn', barmode='group',
                   title='Churn por Gênero')
fig.show()

- Não tem nenhum destaque no churn para diferentes generos.

### Analise financeira

Monthly charge

In [9]:
fig = px.box(dados, x='Churn', y='MonthlyCharges',
             title='Monthly Charges por Churn')
fig.show()

# Violin plot para ver distribuição
fig = px.violin(dados, x='Churn', y='MonthlyCharges',
                title='Distribuição de Monthly Charges')
fig.show()

- A cobrança mensal tem uma tendencia maior ao churn.
- Programas de fidelidade, e descontos em pacotes com mais meses no contrato diminuem o churn.

Total charge

In [ ]:
fig = px.box(dados, x='Churn', y='TotalCharges',
             title='Total Charges por Churn')
fig.show()

- Indicam novamente a possibilidade de churn nos primeiros meses de contrato.

### Analise dos serviços contratados

In [10]:
# Lista de serviços
services = ['PhoneService', 'InternetService', 'StreamingTV', 
            'StreamingMovies', 'OnlineSecurity', 'OnlineBackup', 
            'DeviceProtection', 'TechSupport']

fig = make_subplots(
    rows=4, 
    cols=2,
    subplot_titles=services,
    vertical_spacing=0.1,
    horizontal_spacing=0.08
)

for i, service in enumerate(services):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    # Calcular percentuais
    temp_df = dados.groupby([service, 'Churn']).size().reset_index(name='count')
    total_by_service = temp_df.groupby(service)['count'].transform('sum')
    temp_df['percentage'] = (temp_df['count'] / total_by_service * 100).round(1)
    
    # Criar gráfico com percentuais
    bar_fig = px.bar(
        temp_df, 
        x=service, 
        y='percentage', 
        color='Churn',
        color_discrete_map={'Yes': 'red', 'No': 'green'},
        barmode='group',
        text='percentage'
    )
    
    # Ajustar textos das barras
    bar_fig.update_traces(
        texttemplate='%{text}%',
        textposition='outside'
    )
    
    for trace in bar_fig.data:
        fig.add_trace(trace, row=row, col=col)

fig.update_layout(
    height=1400,
    width=1000,
    title_text="Churn por Serviços Contratados (%)",
    title_x=0.5,
    showlegend=True
)

# Configurar eixos Y para mostrar percentuais
for i in range(1, 9):
    fig.update_yaxes(
        title_text="Percentual %", 
        row=(i+1)//2, 
        col=(i%2)+1 if i%2!=0 else 2,
        range=[0, 100]  # Fixar escala de 0 a 100%
    )

fig.show()

Tipo de contrato

In [11]:
fig = px.histogram(dados, x='Contract', color='Churn', barmode='group',
                   title='Churn por Tipo de Contrato')
fig.show()

- Contratos mês-a-mês, que realizam a cobrança a cada mês, tem uma tendência maior de churn.

### Analise multivariada

In [12]:
# Selecionar apenas variáveis numéricas
numeric_cols = dados.select_dtypes(include=[np.number]).columns
corr_matrix = dados[numeric_cols].corr()

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.columns.tolist(),
    annotation_text=corr_matrix.round(2).values,
    showscale=True
)
fig.update_layout(title='Matriz de Correlação')
fig.show()

In [ ]:
def cramers_v(x, y):
    """Calcula Cramér's V para duas variáveis categóricas"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

def mixed_correlation_matrix(df):
    """Cria matriz de correlação para variáveis mistas"""
    # Selecionar todas as colunas relevantes
    all_cols = df.columns.tolist()
    corr_matrix = pd.DataFrame(index=all_cols, columns=all_cols)
    
    for col1 in all_cols:
        for col2 in all_cols:
            if col1 == col2:
                corr_matrix.loc[col1, col2] = 1.0
            else:
                # Se ambas são numéricas, usar Pearson
                if np.issubdtype(df[col1].dtype, np.number) and np.issubdtype(df[col2].dtype, np.number):
                    corr_matrix.loc[col1, col2] = df[col1].corr(df[col2])
                # Se uma é numérica e outra categórica, usar Point-Biserial aproximado
                elif np.issubdtype(df[col1].dtype, np.number) or np.issubdtype(df[col2].dtype, np.number):
                    # Converter categórica para numérica temporariamente
                    temp_df = df[[col1, col2]].copy()
                    for col in [col1, col2]:
                        if temp_df[col].dtype == 'object':
                            temp_df[col] = pd.Categorical(temp_df[col]).codes
                    corr_matrix.loc[col1, col2] = temp_df[col1].corr(temp_df[col2])
                else:
                    # Ambas categóricas - usar Cramér's V
                    try:
                        corr_matrix.loc[col1, col2] = cramers_v(df[col1], df[col2])
                    except:
                        corr_matrix.loc[col1, col2] = 0
    
    return corr_matrix.astype(float)

# Método simplificado - converter tudo para numérico primeiro
def simple_mixed_correlation(df):
    """Método simplificado: converte tudo para numérico e calcula correlação"""
    df_corr = df.copy()
    
    # Converter todas as colunas categóricas
    for col in df_corr.select_dtypes(include=['object', 'category']).columns:
        if df_corr[col].nunique() == 2:
            # Binária
            df_corr[col] = pd.Categorical(df_corr[col]).codes
        else:
            # Múltiplas categorias - usar codes
            df_corr[col] = pd.Categorical(df_corr[col]).codes
    
    return df_corr.corr()

# Usar o método simplificado
corr_matrix = simple_mixed_correlation(dados)

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns.tolist(),
    y=corr_matrix.columns.tolist(),
    annotation_text=corr_matrix.round(2).values,
    showscale=True,
    colorscale='RdBu'
)
fig.update_layout(
    title='Matriz de Correlação - Todas as Variáveis (Convertidas)',
    width=1500,
    height=1500
)
fig.show()

Correlações positivas
- correlação de tenure e contrato: 0.67 (quanto maior o tempo de contrato mais fidelizado tende a ficar o cliente).

Correlações negativas
- correlação de churn com tenure bem baixa: - 0.35 (indica quanto maior o periodo de assinatura menor o churn).
- correlação de Senioridade com segurança online baixa: - 0.13 (indica uma menor propenção a pessoas mais velhas aderirem esse serviço).
- correlação de senioridade com tech suporte baixa: - 0.15 (indioca uma menor propenção a pessoas mais velhas aderirem esse serviço).
- correlação de churn com partner baixa : - 0.15 (indica que pessoas casadas tem uma menor tendencia de churn).
- correlação de churn com parents baixa : - 0.15 (indica que pessoas com familia tem uma menor tendencia de churn).
- correlação de churn com varios serviços foram negativos: indica quanto mais serviço uma pessoa assina, menor a propenção ao churn.

### Scatter plot

In [ ]:
fig = px.scatter(dados, x='MonthlyCharges', y='TotalCharges', 
                 color='Churn', size='tenure',
                 hover_data=['Contract', 'PhoneService', 'InternetService', 'StreamingTV', 
            'StreamingMovies', 'OnlineSecurity', 'OnlineBackup', 
            'DeviceProtection', 'TechSupport'],
                 title='Monthly vs Total Charges (colorido por Churn)')
fig.show()

Payment method vs churn

In [13]:
fig = px.histogram(dados, x='PaymentMethod', color='Churn', barmode='group',
                   title='Churn por Método de Pagamento')
fig.show()

- Eletronic check possui o maior churn nos metodos de pagamentos (45%)

Dependents Churn

In [ ]:
fig = px.histogram(dados, x='Dependents', color='Churn', barmode='group',
                   title='Churn por Ter Dependentes')
fig.show()

Partners churn

In [ ]:
fig = px.histogram(dados, x='Partner', color='Churn', barmode='group',
                   title='Churn por Ter Parceiro')
fig.show()

- Solteiros tem uma leve tendencia a mais ao churn.

Paperlessbilling

In [ ]:
fig = px.histogram(dados, x='PaperlessBilling', color='Churn', barmode='group',
                   title='Churn por cobrança eletronica')
fig.show()

: 

### Outros gráficos

In [ ]:
# Criar subplots para análise rápida
categories = ['Contract', 'PhoneService', 'InternetService', 'StreamingTV', 
            'StreamingMovies', 'OnlineSecurity', 'OnlineBackup', 
            'DeviceProtection', 'TechSupport']

fig = make_subplots(rows=3, cols=3, subplot_titles=categories)

for i, cat in enumerate(categories):
    row = i // 3 + 1
    col = i % 3 + 1
    
    churn_rate = dados.groupby(cat)['Churn'].mean().reset_index()
    fig.add_trace(
        px.bar(churn_rate, x=cat, y='Churn').data[0],
        row=row, col=col
    )

fig.update_layout(height=600, title_text="Taxa de Churn por Categoria")
fig.show()

### 4.Analises numericas

In [ ]:
dados.head()

In [ ]:
dados['Contract'].value_counts(normalize=True)

In [ ]:
dados['PaperlessBilling'].value_counts(normalize=True)

In [ ]:
pagamento_eletronico = dados[dados['PaperlessBilling'] == 'No']
pagamento_no_papel = dados[dados['PaperlessBilling'] == 'Yes']

In [ ]:
pagamento_eletronico['Churn'].value_counts(normalize=True)

In [ ]:
pagamento_no_papel['Churn'].value_counts(normalize=True)

In [7]:
dados['InternetService'] = dados['InternetService'].replace(['Fiber optic', 'DSL'], 'Yes')

In [ ]:
servicos = 0
services = ['PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

In [8]:
dados['InternetService'].value_counts()

InternetService
Yes    5517
No     1526
Name: count, dtype: int64

In [12]:
# Contagem de 'yes' em cada coluna de serviço
contagem_por_servico = dados[services].eq('Yes').sum()

print("Contagem de 'Yes' por serviço:")
print(contagem_por_servico)

# Em formato de porcentagem
total_clientes = len(dados)
porcentagem_por_servico = (dados[services].eq('Yes').sum() / total_clientes * 100).round(2)

print("\nPorcentagem de 'Yes' por serviço:")
print(porcentagem_por_servico)

Contagem de 'Yes' por serviço:
PhoneService        6361
MultipleLines       2971
InternetService     5517
OnlineSecurity      2019
OnlineBackup        2429
DeviceProtection    2422
TechSupport         2044
StreamingTV         2707
StreamingMovies     2732
dtype: int64

Porcentagem de 'Yes' por serviço:
PhoneService        90.32
MultipleLines       42.18
InternetService     78.33
OnlineSecurity      28.67
OnlineBackup        34.49
DeviceProtection    34.39
TechSupport         29.02
StreamingTV         38.44
StreamingMovies     38.79
dtype: float64


In [15]:
# Adicionar uma coluna com o total de serviços assinados por cliente
dados['total_servicos'] = dados[services].eq('Yes').sum(axis=1)

# Ver a distribuição
print("Distribuição do número de serviços por cliente:")
print(dados['total_servicos'].value_counts().sort_index())
# Estatísticas descritivas
print(f"\nEstatísticas dos serviços por cliente:")
print(f"Média: {dados['total_servicos'].mean():.2f}")
print(f"Mediana: {dados['total_servicos'].median()}")
print(f"Máximo: {dados['total_servicos'].max()}")
print(f"Mínimo: {dados['total_servicos'].min()}")

Distribuição do número de serviços por cliente:
total_servicos
1    1264
2     859
3     846
4     965
5     922
6     908
7     676
8     395
9     208
Name: count, dtype: int64

Estatísticas dos serviços por cliente:
Média: 4.15
Mediana: 4.0
Máximo: 9
Mínimo: 1


In [20]:
dados['total_servicos'].mean()

np.float64(4.146244498083203)

In [18]:
dados['valor_servico'] = dados['MonthlyCharges'] / dados['total_servicos']

In [21]:
dados['valor_servico'].mean()

np.float64(17.285098890827097)

In [19]:
dados.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,total_servicos,valor_servico
0,7569-NMZYQ,Female,0,Yes,Yes,72,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,9,13.194444
1,8984-HPEMB,Female,0,No,No,71,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,9,13.183333
2,5989-AXPUC,Female,0,Yes,No,68,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No,9,13.177778
3,5734-EJKXG,Female,0,No,No,61,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,118.60,7365.70,No,9,13.177778
4,8199-ZLLSA,Male,0,No,No,67,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),118.35,7804.15,Yes,9,13.150000


In [22]:
dados['MonthlyCharges'].mean()

np.float64(64.76169246059918)

In [23]:
dados['TotalCharges'].mean()

np.float64(2283.3004408418656)